# Alguns métodos de resolução de sistemas lineares

# Método de eliminação de Gauss - pivotação parcial

###### Neste método, ordenamos as linhas da matriz de modo que a diagonal principal sempre tenha o maior valor (em módulo) na coluna. Ajuda a resolver problemas com arredondamento, mas dificulta um pouco a construção Por isso, vamos construir as funções aos poucos e finalizar tudo numa única função.

In [129]:
import numpy as np
import pandas as pd

In [11]:
n = 3 ## tamanho da matriz de exemplo

In [95]:
x = np.zeros((n, 1)) ##esse vetor vai conter as respostas do sistema

In [104]:
M = np.zeros((n,n+1)) ##seta uma matriz de exemplo; a coluna sempre tem que ter dimensão n+1 pois ela recebe
#tambem o vetor de resposta. Vamos a seguir popular a nossa matriz com um exemplo simples de sistema possível e determinado
### como exemplo de aplicação do algoritmo de Gauss sem pivotação 

In [109]:
M[0][0] = -0.319
M[0][1] = 0.884
M[0][2] = 0.279
M[0][3] = 0
M[1][0] = 0.421
M[1][1] = 0.784
M[1][2] = -0.207
M[1][3] = 0
M[2][0] = 0.448
M[2][1] = 0.832
M[2][2] = 0.193
M[2][3] = 1

### Pivotação parcial
#### A função abaixo vai encontrar o indice (de linha) do maior valor em módulo na coluna que queremos pivotar, a partir da linha i == j

In [119]:
def max_idx(M, n, j): ###
    max_ = 0
    idx = 0
    for i in range(j,n):
        if abs(M[i,j]) >= max_:
            max_ = M[i,j]
            idx = i
    return idx

In [77]:
M

array([[-0.319,  0.884,  0.279,  0.   ],
       [ 0.421,  0.784, -0.207,  0.   ],
       [ 0.448,  0.832,  0.193,  1.   ]])

In [78]:
teste = pivota(M, n, 0)

### A função abaixo faz a troca de linhas, de acordo com a coluna j em que estamos

In [117]:
def pivota(M, n, j):
    idx_max = max_idx(M, n, j)
    temp = M[j,:].copy()
    M[j,:] = M[idx_max,:]
    M[idx_max,:] = temp
    return M

### Escalonamento da matriz

In [118]:
def triangula(M, n, j):   ###
                         ###esta função traz o indice da coluna em que estamos, a matriz, e o tamanho da matriz de coeficientes
    pivo = M[j,j]         ### ela vai escalonar a linha a partir do indice da coluna
    for i in range(j+1,n):
        M[i,:] = M[i,:] - (M[i,j]/pivo)*M[j,:]
    return M

### Juntando tudo numa funçãozinha só

In [120]:
###vamos juntar tudo numa função só para mostrar que funciona
def pivotacao_parcial(M,n):
    for j in range(n):
        M = pivota(M, n, j)
        M = triangula(M, n, j)
    return M

### A implementação acima funciona, mas tem funçõezinhas em separado. Conseguimos juntar tudo em uma só: encontrar o maior valor em módulo, trocar as linhas, e escalonar :)


In [130]:
def pivotacao_parcial_tot(M, n):
    for j in range(n):
        max_ = 0
        idx = 0
        for i in range(j,n):
            if abs(M[i,j]) >= max_:
                max_ = M[i,j]
                idx = i
        i=0
        temp = M[j,:].copy()
        M[j,:] = M[idx,:]
        M[idx,:] = temp
        pivo = M[j,j]         ### ela vai escalonar a linha a partir do indice da coluna
        for i in range(j+1,n):
            M[i,:] = M[i,:] - (M[i,j]/pivo)*M[j,:]
    return M

In [131]:
n = 4
M = np.zeros((n,n+1))
M[0][0] = 1
M[0][1] = 2
M[0][2] = 3
M[0][3] = 4
M[0][4] = 5

M[1][0] = 2
M[1][1] = 0
M[1][2] = 13
M[1][3] = 11
M[1][4] = 0

M[2][0] = 14
M[2][1] = 21
M[2][2] = 9
M[2][3] = 5
M[2][4] = -3



M[3][0] = 5
M[3][1] = 4
M[3][2] = 3
M[3][3] = 2
M[3][4] = 1


In [124]:
M

array([[ 1.,  2.,  3.,  4.,  5.],
       [ 2.,  0., 13., 11.,  0.],
       [14., 21.,  9.,  5., -3.],
       [ 5.,  4.,  3.,  2.,  1.]])

In [111]:
teste = pivotacao_parcial_tot(M, 4)

## Retrossubstituição (ou, atribuição das soluções do sistema)

In [113]:
## começamos com o caso de i = n (andamos de baixo pra cima na matriz)
x[n-1] = M[n-1, n]/M[n-1, n-1]

In [114]:
for i in range(1, n): ###restringimos os casos a partir de n-1. o indexador do python ja fa isso naturalmente [1,n) ==[1,n-1]
    termo_ind = M[n-i-1,n]/M[n-i-1,n-i-1] ###termo independente da equacao
    soma = 0
    for j in range(n-i, n):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
        soma += ((M[n-i-1,j])/(M[n-i-1,n-i-1]))*x[j]
    x[n-i-1] = termo_ind - soma          ####o vetor de respostas vai sendo alimentado de tras pra frente tambem

In [115]:
pd.DataFrame(x)

,0
0,0.8
1,-0.2
2,-3.0
3,3.4


### Vamos jogar tudo isso numa função também 

In [128]:
def solucao(M, n):
    
    x = np.zeros((n, 1)) ###vetor que vai armazenar a resposta
    
    ## começamos com o caso de i = n (andamos de baixo pra cima na matriz)
    x[n-1] = M[n-1, n]/M[n-1, n-1]
    
    for i in range(1, n): ###restringimos os casos a partir de n-1. o indexador do python ja fa isso naturalmente [1,n) ==[1,n-1]
        termo_ind = M[n-i-1,n]/M[n-i-1,n-i-1] ###termo independente da equacao
        soma = 0
        for j in range(n-i, n):           ###esta parte do loop é a mais crucial de ser entendida; escreva numa folhinha de papel o que ela faz que fica facil de enxergar
            soma += ((M[n-i-1,j])/(M[n-i-1,n-i-1]))*x[j]
        x[n-i-1] = termo_ind - soma          ####o vetor de respostas vai sendo alimentado de tras pra frente tambem
    
    return x
    

In [132]:
A = pivotacao_parcial_tot(M, n)

In [133]:
A

array([[14.        , 21.        ,  9.        ,  5.        , -3.        ],
       [ 0.        , -3.5       , -0.21428571,  0.21428571,  2.07142857],
       [ 0.        ,  0.        , 11.89795918, 10.10204082, -1.34693878],
       [ 0.        ,  0.        ,  0.        ,  1.69811321,  5.77358491]])

In [134]:
sol = solucao(A, n)
sol

array([[ 0.8],
       [-0.2],
       [-3. ],
       [ 3.4]])

### Verificando se resultou na solução correta:

In [136]:
np.dot(A[:,0:4],sol) #Sim!

array([[-3.        ],
       [ 2.07142857],
       [-1.34693878],
       [ 5.77358491]])